<a href="https://colab.research.google.com/github/yiwenwangANU/MachineLearningTools/blob/main/tf_keras_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
inputs = layers.Input(shape=(None, None, 3))
processed = layers.RandomCrop(width=32, height=32)(inputs)
conv = layers.Conv2D(filters=2, kernel_size=3)(processed)
pooling = layers.GlobalAveragePooling2D()(conv)
feature = layers.Dense(10)(pooling)

In [ ]:
full_model = tf.keras.Model(inputs, feature)

In [ ]:
full_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 random_crop (RandomCrop)    (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 2)         56        
                                                                 
 global_average_pooling2d (G  (None, 2)                0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 10)                30        
                                                                 
Total params: 86
Trainable params: 86
Non-trainable params: 0
_________________________________________________________________

In [ ]:
backbone = tf.keras.Model(processed, conv)
activations = tf.keras.Model(conv, feature)
backbone.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 2)         56        
                                                                 
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)

model = MyModel()
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

In [ ]:
x = tf.random.normal([2, 2])
print(x)
model(x)
#model.build(input_shape=(32, 100))
model.summary()

tf.Tensor(
[[ 0.00884323  0.32622874]
 [ 1.9649651  -1.4598855 ]], shape=(2, 2), dtype=float32)
Model: "my_model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            multiple                  12        
                                                                 
 dense_18 (Dense)            multiple                  25        
                                                                 
Total params: 37
Trainable params: 37
Non-trainable params: 0
_________________________________________________________________


#SparseCategoricalCrossentropy()

In [ ]:
y_true = [1, 2]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
loss = tf.keras.losses.SparseCategoricalCrossentropy()
loss(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=1.1769392>

In [ ]:
y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]

loss = tf.keras.losses.CategoricalCrossentropy()
loss(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=1.1769392>

In [ ]:
y_true = [1, 2]
y_pred = [[5, 95, 0], [1, 8, 1.9]] 
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) #Whether y_pred is expected to be a logits tensor
# logit map (0, 1) to (-info, +info), sigmoid map (-info, +info) to (0, 1)
# no need acitvation='softmax' in Dense layer
loss(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=3.051575>

#Sparse Tensor

In [ ]:
x = tf.SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
x = tf.sparse.to_dense(x)
x

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[1, 0, 0, 0],
       [0, 0, 2, 0],
       [0, 0, 0, 0]], dtype=int32)>

In [ ]:
x = tf.sparse.from_dense(x)
x

In [ ]:
tf.sparse.to_dense(x.with_values([10, 20]))

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[10,  0,  0,  0],
       [ 0,  0, 20,  0],
       [ 0,  0,  0,  0]], dtype=int32)>

#Random_normal_initializer

In [ ]:
init = tf.random_normal_initializer(mean=1., stddev=2.)
x = tf.Variable(init(shape=[2, 2],
                     dtype=tf.float32))
x

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[ 1.6370089 , -1.5740042 ],
       [-0.12616086, -0.47901666]], dtype=float32)>

#tf.random.categorical

In [8]:
x = tf.math.log([[1, 0.33, 0.33]])
x

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 0.       , -1.1086626, -1.1086626]], dtype=float32)>

In [14]:
samples = tf.random.categorical(x, 10)
samples

<tf.Tensor: shape=(1, 10), dtype=int64, numpy=array([[0, 2, 1, 2, 0, 0, 2, 0, 0, 2]])>

#Subclassing API

##Setup

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [ ]:
x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.08597028 -0.05656605  0.09882867 -0.00835464]
 [-0.08597028 -0.05656605  0.09882867 -0.00835464]], shape=(2, 4), dtype=float32)


In [ ]:
linear_layer.weights

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[-0.02022812, -0.03956595,  0.01768934, -0.03593931],
        [-0.06574216, -0.0170001 ,  0.08113934,  0.02758468]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)


tf.Tensor(
[[0.06384281 0.05322126 0.05991554 0.0113938 ]
 [0.06384281 0.05322126 0.05991554 0.0113938 ]], shape=(2, 4), dtype=float32)


##Non-trainable weights

In [ ]:
class ComputeSum(keras.layers.Layer):
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)), trainable=False)

    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total

In [ ]:
x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())


[2. 2.]
[4. 4.]


In [ ]:
my_sum.weights

[<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([4., 4.], dtype=float32)>]

##When the input shape is unknown

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [ ]:
x = tf.ones((2, 2))
linear_layer = Linear(4)

In [ ]:
y = linear_layer(x) #  __call__() method of your layer will automatically run build the first time it is called. 
y

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 0.04992267, -0.02960005,  0.11418498,  0.01088237],
       [ 0.04992267, -0.02960005,  0.11418498,  0.01088237]],
      dtype=float32)>

##Layer Block

In [ ]:
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


In [ ]:
mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 2, 2))) 
y

<tf.Tensor: shape=(3, 2, 1), dtype=float32, numpy=
array([[[-7.6673925e-05],
        [-7.6673925e-05]],

       [[-7.6673925e-05],
        [-7.6673925e-05]],

       [[-7.6673925e-05],
        [-7.6673925e-05]]], dtype=float32)>

##add_loss() method to create your own regularization & training loss

In [ ]:
# A layer that creates an activity regularization loss
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

In [ ]:
class OuterLayer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularizationLayer(1e-2)

    def call(self, inputs):
        return self.activity_reg(inputs)

In [ ]:
layer = OuterLayer()
layer.losses

[]

In [ ]:
x = layer(tf.ones([2, 2]))
layer.losses, x

([<tf.Tensor: shape=(), dtype=float32, numpy=0.04>],
 <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 1.],
        [1., 1.]], dtype=float32)>)

##add_metric

In [ ]:
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()

    def call(self, targets, logits, sample_weights=None):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        loss = self.loss_fn(targets, logits, sample_weights)
        self.add_loss(loss)

        # Log accuracy as a metric and add it
        # to the layer using `self.add_metric()`.
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name="accuracy")

        # Return the inference-time prediction tensor (for `.predict()`).
        return tf.nn.softmax(logits)


In [ ]:
layer = LogisticEndpoint()

targets = tf.ones((2, 2))
logits = tf.ones((2, 2))
y = layer(targets, logits)


In [ ]:
y

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.5, 0.5],
       [0.5, 0.5]], dtype=float32)>

In [ ]:
layer.metrics[0].result()

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [ ]:
layer.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.3132617>]